In [282]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from scraping import *
import random
from collections import defaultdict 
from ast import literal_eval

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Base Links

All Sides labels articles as stories when it addresses a news topic with 3 news sources linked (mostly from left, center, right).   
We pass in the sitemap urls and use custom defined functions to get a list of all the links containing stories.
Stories can be identified by looking at the url for a '/story/' component.

In [ ]:
sitemap_url = ['https://www.allsides.com/sitemap.xml?page=1',\
                 'https://www.allsides.com/sitemap.xml?page=2']

story_links = spy.allsides_sitemap_story_parser(sitemap_url)

In [ ]:
story_links[10:20]

### Scrape each story link

Now that we have a list containing links to all the stories (as on 11-20-2020), we can scrape every individual story page to get the story title, data, summary and the news sources under them.

In [306]:
stories_df = allsides_story_parser(story_links, filename = "../Data/news_while_scraping_save.csv", verbose = 25)

25 https://www.allsides.com/story/ross-perot-former-independent-presidential-candidate-dead-89 Ross Perot, Former Independent Presidential Candidate, Dead at 89 July 9th, 2019
50 https://www.allsides.com/story/bernie-sanders-defends-staff-wages-after-complaints Bernie Sanders Defends Staff Wages After Complaints July 21st, 2019
75 https://www.allsides.com/story/joe-biden-swings-back-challengers Joe Biden Swings Back At Challengers August 1st, 2019
100 https://www.allsides.com/story/hong-kong-protesters-clash-police-airport Hong Kong: Protesters Clash With Police at Airport August 13th, 2019
125 https://www.allsides.com/story/facebook-bans-ads-epoch-times Facebook Bans Ads From The Epoch Times August 24th, 2019
150 https://www.allsides.com/story/pro-eu-conservatives-vote-against-johnson-parliament Pro-EU Conservatives Vote Against Johnson in Parliament September 3rd, 2019
175 https://www.allsides.com/story/doj-rejects-mccabes-appeal-avoid-charges DOJ Rejects McCabe's Appeal to Avoid Cha

In [258]:
# new_df['news_sources'] = new_df.news_sources.map(literal_eval)
new_df['summary'] = new_df.summary.map(literal_eval)

In [308]:
stories_df.to_csv("../Data/allsides_stories.csv", index=False)